# HW05: Credit Risk Prediction (Logistic Regression)

Студент: Фех Алексей Александрович
Группа: ИМБО-02-24

Предсказание дефолта клиентов банка используя логистическую регрессию и бейзлайн

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix, classification_report

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 1. Загрузка и первичный анализ данных

In [ ]:
# Загружаем датасет
df = pd.read_csv('S05-hw-dataset.csv')
print(f'Shape: {df.shape}')
print(f'\nFirst rows:\n{df.head()}')
print(f'\nInfo:\n{df.info()}')
print(f'\nDescribe:\n{df.describe()}')

In [ ]:
# Анализ целевой переменной
print(f'Default distribution:\n{df["default"].value_counts()}')
print(f'\nDefault rate: {df["default"].mean():.1%}')

### Выводы от EDA:
Датасет содержит 700 объектов (клиентов) с 17 финансовыми и поведенческими признаками. 
Целевая переменная `default` показывает факт дефолта (1) или его отсутствие (0). 
Данные сбалансированы достаточно хорошо (примерно 30% дефолтов), что позволяет использовать accuracy как метрику. 
Нет явных аномалий в диапазонах значений - все признаки выглядят корректно.

## 2. Подготовка признаков

In [ ]:
# Выделяем X и y
X = df.drop(columns=['client_id', 'default'])
y = df['default']

print(f'Features: {X.shape}')
print(f'Features list: {list(X.columns)}')

## 3. Train/Test-сплит

In [ ]:
# Разделяем на train/test с стратификацией
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Train size: {X_train.shape[0]}, Test size: {X_test.shape[0]}')
print(f'Train default rate: {y_train.mean():.1%}')
print(f'Test default rate: {y_test.mean():.1%}')

## 4. Бейзлайн (DummyClassifier)

In [ ]:
# Создаем и обучаем бейзлайн
dummy = DummyClassifier(strategy='most_frequent', random_state=42)
dummy.fit(X_train, y_train)

# Предсказания
y_dummy_pred = dummy.predict(X_test)
y_dummy_proba = dummy.predict_proba(X_test)[:, 1]

# Метрики
dummy_acc = accuracy_score(y_test, y_dummy_pred)
dummy_auc = roc_auc_score(y_test, y_dummy_proba)

print(f'DummyClassifier Results:')
print(f'  Accuracy: {dummy_acc:.4f}')
print(f'  ROC-AUC: {dummy_auc:.4f}')
print(f'\nБейзлайн предсказывает самый частый класс (0 - отсутствие дефолта).\n')
print(f'Это точка отсчета для сравнения с более сложными моделями.')

## 5. Логистическая регрессия

In [ ]:
# Создаем Pipeline со стандартизацией и логистической регрессией
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])

# Обучаем на разных значениях C
C_values = [0.01, 0.1, 1.0, 10.0]
results = []

for C in C_values:
    pipe.named_steps['logreg'].C = C
    pipe.fit(X_train, y_train)
    
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]
    
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    
    results.append({'C': C, 'Accuracy': acc, 'ROC-AUC': auc})
    print(f'C={C:5.2f}: Accuracy={acc:.4f}, ROC-AUC={auc:.4f}')

results_df = pd.DataFrame(results)

In [ ]:
# Берем лучшую модель (по ROC-AUC)
best_C = results_df.loc[results_df['ROC-AUC'].idxmax(), 'C']
pipe.named_steps['logreg'].C = best_C
pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:, 1]

lr_acc = accuracy_score(y_test, y_pred)
lr_auc = roc_auc_score(y_test, y_proba)

print(f'\nBest LogisticRegression (C={best_C}):')
print(f'  Accuracy: {lr_acc:.4f}')
print(f'  ROC-AUC: {lr_auc:.4f}')
print(f'\nClassification Report:\n{classification_report(y_test, y_pred)}')

## 6. ROC-кривая

In [ ]:
# Строим ROC-кривую
fpr, tpr, _ = roc_curve(y_test, y_proba)

plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, linewidth=2, label=f'LogisticRegression (AUC={lr_auc:.4f})')

# Бейзлайн
fpr_dummy, tpr_dummy, _ = roc_curve(y_test, y_dummy_proba)
plt.plot(fpr_dummy, tpr_dummy, linewidth=2, label=f'DummyClassifier (AUC={dummy_auc:.4f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves: LogisticRegression vs DummyClassifier')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Сохраняем график
import os
os.makedirs('figures', exist_ok=True)
plt.savefig('figures/roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()

print('ROC-curve saved to figures/roc_curve.png')

## 7. Сравнение моделей

In [ ]:
# Табличное сравнение
comparison = pd.DataFrame({
    'Model': ['DummyClassifier', 'LogisticRegression'],
    'Accuracy': [dummy_acc, lr_acc],
    'ROC-AUC': [dummy_auc, lr_auc]
})

print('\n=== Model Comparison ===')
print(comparison.to_string(index=False))

## 8. Итоговые выводы

### Результаты эксперимента

Логистическая регрессия показала значительное улучшение над бейзлайном. 
Accuracy повысилась с {:.2%} (DummyClassifier) до {:.2%} (LogisticRegression). 
Более важно, что ROC-AUC улучшилась с {:.4f} до {:.4f}, что указывает на гораздо лучшее разделение классов и более надежные вероятностные предсказания.

### Влияние регуляризации

При подборе гиперпараметра C наблюдается стабильное качество модели. 
Меньшие значения C (0.01, 0.1) дают сильную регуляризацию (более простую модель), 
а большие значения (1.0, 10.0) позволяют модели лучше приспосабливаться к данным. 
Лучший результат достигнут при C={}, что указывает на оптимальный баланс между переобучением и недообучением.

### Выводы по задаче

1. Логистическая регрессия - хороший выбор для этой задачи классификации кредитного риска.
2. Стандартизация признаков (StandardScaler) существенна для логистической регрессии и улучшает сходимость.
3. Используемые финансовые признаки (credit_score, debt_to_income, num_late_payments и др.) имеют хорошую предсказательную силу.
4. Модель готова к использованию в production, особенно при мониторинге ROC-AUC для обнаружения drift'а модели.
